In [1]:
import os
import gmaps
import gmaps.datasets
import pandas as pd

In [2]:
%pylab inline

# See the https://jupyter-gmaps.readthedocs.io/en/latest/install.html for installation
# set GOOGLE_API_KEY in your environment
gmaps.configure(api_key=os.environ["GOOGLE_API_KEY"])

Populating the interactive namespace from numpy and matplotlib


### Loading Data

In [3]:
date_cols = ['pickup_datetime', 'dropoff_datetime',]
ride_data = pd.read_csv('ride_data.csv', parse_dates=date_cols)

In [4]:
pickup_locations = ride_data[['pickup_latitude', 'pickup_longitude']]
locations = pickup_locations.sample(frac=0.5).values

In [5]:
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations))
fig

Figure(layout=FigureLayout(height='420px'))